In [1]:
import sys
import os
sys.path.append(os.path.abspath('../..'))

In [2]:
from src.modules.pdf_interpreter import text_from_pdf, get_transactions
from src.modules.helper_fns import *

In [3]:
import pandas as pd
import re

In [4]:
account_names = read_all_account_folder_names()
account_names

['account_placeholder', 'Chequing', 'Credit', 'MoneyMaster']

In [5]:
overall_df = pd.DataFrame()
for account_name in ['Chequing', 'Credit']:
    pdf_files = read_all_files(account_name)
    for pdf_file in pdf_files:
        print(pdf_file)
        pdf_file_atts = grab_pdf_name_attributes(pdf_file)
        pdf_file_path = process_import_path(pdf_file, account_name)
        lines = text_from_pdf(pdf_file_path, x_tolerance=2, init_y_top=440, reg_y_top=210)
        transactions= get_transactions(lines, account_name)
        temp_df = pd.DataFrame(transactions)
        temp_df[['year']] = pdf_file_atts['year']
        temp_df[['Account']] = account_name
        overall_df = pd.concat([temp_df, overall_df], ignore_index=True)

APR 2022_eStatement.pdf
August 2022 e-Statement .pdf
December 2021 e-Statement .pdf
December 2022 e-Statement .pdf
February 2022 e-Statement .pdf
February 2023 e-Statement .pdf
January 2022 e-Statement .pdf
January 2023 e-Statement .pdf
July 2022 e-Statement .pdf
June 2022 e-Statement .pdf
March 2022 e-Statement.pdf
May 2022 e-Statement .pdf
November 2021 e-Statement .pdf
November 2022 e-Statement .pdf
October 2021 e-Statement .pdf
October 2022 e-Statement .pdf
September 2021 e-Statement .pdf
September 2022 e-Statement .pdf
April 2022 e-Statement.pdf
August 2022 e-Statement.pdf
December 2021 e-Statement.pdf
December 2022 e-Statement.pdf
February 2022 e-Statement.pdf
February 2023 e-Statement.pdf
January 2022 e-Statement.pdf
January 2023 e-Statement.pdf
July 2022 e-Statement.pdf
June 2022 e-Statement.pdf
March 2022 e-Statement.pdf
March 2023 e-Statement.pdf
May 2022 e-Statement.pdf
November 2021 e-Statement.pdf
November 2022 e-Statement.pdf
October 2021 e-Statement.pdf
October 2022 e-St

In [6]:
overall_df

,Reference #,Transaction Date,Post Date,Details,Amount,Transaction Type,year,Account
0,005,Aug 10,Aug 10,MB-CREDIT CARD/LOC PAY. FROM -,40.91,*****04*1181 MR JA,2022,Credit
1,006,Aug 11,Aug 12,AMZN Mktp CA*BM2420FT3,207.91,WWW.AMAZON.CA ON,2022,Credit
2,007,Aug 12,Aug 15,A&W 4965 NEPEANON,7.10,For al,2022,Credit
3,008,Aug 12,Aug 15,OCT-CONFEDERATION-PARLIAM OTTAWAON,20.00,most,2022,Credit
4,009,Aug 13,Aug 15,Subway 4618Ottawa ON,14.10,point,2022,Credit
...,...,...,...,...,...,...,...,...
892,NaN,Apr 8,NaN,Credit Card,123.41,MB-Transfer to,2022,Chequing
893,NaN,Apr 11,NaN,Cra (Rev) Tax Amount Owing,200.00,TS Bill payment,2022,Chequing
894,NaN,Apr 14,NaN,Opos Uber Canada Toronto ONCA,23.95,Point of sale purchase,2022,Chequing
895,NaN,Apr 14,NaN,Opos Uber Canada Toronto ONCA,1.05,Point of sale purchase,2022,Chequing


In [7]:
import re
import calendar

def __extract_month(input_string):
    months = '|'.join(f'.*{m}.*' for m in calendar.month_abbr[1:])
    month_match = re.match(f'.*({months}).*', input_string, re.IGNORECASE)
    if month_match:
        matched_text = month_match.group(1)
        for i, abbr in enumerate(calendar.month_abbr[1:], 1):
            if abbr.lower() in matched_text.lower():
                return calendar.month_name[i]

def __interpret_name(pdf_name):
    pdf_name = pdf_name.replace('.pdf', '')
    str_atts = re.split(r'[ _]', pdf_name)\
    
    result = {'month':'NA', 'year': 'NA'}
    for str_attribute in str_atts:
        month_extracted = __extract_month(str_attribute)
        if month_extracted is not None:
            result['month'] = month_extracted
        elif re.match(r'^\d{4}$', str_attribute) and 1000 <= int(str_attribute) <= 9999:
            result['year'] = str_attribute

    return result

def __process_cache_path(pdf_file, parent_account_name):
    pdf_atts = __interpret_name(pdf_file)
    full_path = "../../bank_statements/cached_data/" + parent_account_name + pdf_atts["month"] + "_" + pdf_atts['year'] + ".csv"
    return full_path

In [8]:
asd = __process_cache_path("decm 2022 e-Statement.pdf", "ROGERT")
asd

'../../bank_statements/cached_data/ROGERTDecember_2022.csv'